In [1]:
import torch 
import matplotlib.pyplot as plt
from utils import * 
from mlmm import *
from einops import rearrange, repeat
import numpy as np

In [2]:
# torch.set_printoptions(precision=2, sci_mode=False, linewidth=120)

In [3]:
# singular smooth kernel 
l = 13
n = 2**l - 1
xmin = -1
xmax = 1
h = (xmax - xmin)/(n+1)

xh = torch.linspace(-1, 1, n+2)[1:-1][None][None] 

# input function
uh = 1-xh**2

# kernel function
gh_X, gh_Y = torch.meshgrid(xh[0,0], xh[0,0])
Khh = torch.log((gh_Y - gh_X).abs()+1e-5)[None][None]

# output function(numeric)
wh = multi_summation(Khh, uh, h)

/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [4]:
rl2 = LpLoss(size_average=False)

MLMM singular smooth case

In [12]:
order = 2
m = 1
k = 4

In [13]:
wh_, KHH, Khh_correction_lst, nbr_idx_lst = SingularSmoothKernelMLMM(uh, Khh, h, k, order, m)
l1_error = L1Norm(wh_, wh)
rl2_error = rl2(wh_, wh)
print("L1 - {:.2e}, rL2 - {:.2e}".format(l1_error, rl2_error))

L1 - 1.76e-03, rL2 - 5.14e-03


In [14]:
%%timeit 
wh = multi_summation(Khh, uh, h)

4.14 ms ± 74.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit
wh_ = SingularSmoothKernelMLMM_(KHH, uh, h, Khh_correction_lst, nbr_idx_lst)
l1_error = L1Norm(wh_, wh)
rl2_error = rl2(wh_, wh)
# print("L1 - {:.2e}, rL2 - {:.2e}".format(l1_error, rl2_error))

2.08 ms ± 27.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
# error_lst = []
# for k in range(1, 12):
#     wh_ = SingularSmoothKernelMLMM(uh, Khh, h, k, order, m)
#     l1_error = L1Norm(wh_[0], wh)
#     print(l1_error)
#     error_lst.append(l1_error)

# plt.plot(np.arange(1,12), error_lst)
# plt.yscale('log')